# SYB MVP Tutorial

This notebook demonstrates how to use the MVP (Minimum Viable Product) version of the Sybil Resistant System.

## Getting Started: The MVP Contract (VouchMinimal)

Let's set up and run the MVP interface!

### Step 1: Import Required Modules and create a random network.

We import the needed modules and create a random network consisting of 8 users.

In [ ]:
# Import the MVP components
from syb_mvp_ui import create_random_mvp_network, SYBMvpUserInterface
from contract_interface_mvp import VouchMinimal

# Create a network with 8 users
contract, users = create_random_mvp_network(num_users=8)

print(f"\n📊 Network created:")
print(f"  - Users: {len(users)}")
print(f"  - Initial vouches: {contract.network.number_of_edges()}")
print(f"  - Seed vouches used: {contract.seed_vouch_count}")

### Step 2: Create the UI 

We will now create the User Interface object in the background. We won't display it just yet.

This allows us to perform scripted actions in Step 3 and have them update the UI object's state.

In [ ]:
import matplotlib.pyplot as plt

# Turn off interactive mode
plt.ioff()

# Create UI object but DO NOT display it yet.
# We will display it at the end.
first_user_addr = list(users.keys())[0]
ui = SYBMvpUserInterface(contract=contract, users=users, current_user_address=first_user_addr)

print("UI object created. We will display it after the scripted tutorial.")

### Initial Network State

Let's see the network after initialization, before we make any changes.

> NOTE: You cannot unvouch for anyone initially since you haven't vouched for anyone yet.

#### Current User

The **Current User** section displays information about the account you're currently using:

- **Name**: The alphabetical name assigned to this user (e.g., Alice, Bob, Charlie)
- **Address**: The Ethereum-style address of the current user account
- **Rank**: The user's rank in the network (lower is better, "DEFAULT" means unranked)
- **Score**: The computed score based on vouching relationships and network structure (higher is better)
- **Outdegree**: Number of users this account has vouched for
- **In-degree**: Number of users who have vouched for this account

#### Network Status

The **Network Status** section provides a comprehensive overview of the entire network:

- **Total Users**: The number of active accounts in the network
- **Total Vouches**: The total number of vouching relationships (edges) in the network
- **User Rankings Table**: A sortable table showing all users with their:
  - **User**: The alphabetical name of each user
  - **Rank**: Current rank (lower is better, "DEFAULT" means the nodes has just been initialized, and "0" means unranked)
  - **Prev Rank**: Previous rank value (shows "-" if no previous rank existed)
  - **Score**: Current computed score
  - **Prev Score**: Previous score value (shows 0 if no previous score existed)
  - **Out**: Number of outgoing vouches (who this user vouches for)
  - **In**: Number of incoming vouches (who vouches for this user)

The table is sorted by score (highest first), and cells are highlighted in yellow when rank or score has changed, making it easy to track network dynamics.

In [ ]:
# Now, we can display various network states using the plotting utilities.
# We import the plotting functions
from plot_utils import show_current_user_status, show_network_status, show_network_graph
# Show the initial states
show_network_status(contract, users, ui)
show_network_graph(contract, users, ui)

## Step 3: Scripting Network Actions

Let's walk through a few examples illustating the actions users can do.

We'll introduce a new user, 'Newbie', and have our first user, 'Alice', vouch for them.

In [ ]:
# We need this function to create a new address
from contract_interface_mvp import generate_mul_eth_addresses

# Get Alice's address (the first user)
alice_addr = list(users.keys())[0]
alice_name = ui._get_display_name(users[alice_addr]['name'])

# 1. Introduce a new node
newbie_addr = generate_mul_eth_addresses(1)[0]
newbie_name = "Newbie"

# Add 'Newbie' to the contract's node list (vouch will do this, but let's be explicit)
# We must add to the UI's user list so it can be displayed
users[newbie_addr] = {
    'name': newbie_name,
    'address': newbie_addr
}

# 2. Make Alice vouch for Newbie
print(f"Action: {alice_name} is vouching for {newbie_name}...")
try:
    contract.vouch(alice_addr, newbie_addr)
    print("...Vouch successful! Displaying updated network state:")

    # Refresh the UI object (for Step 4)
    ui.vouch_target.options = ui._get_vouchable_users()
    ui.unvouch_target.options = ui._get_unvouchable_users()
    ui._update_all()
    
    # Show the current state
    show_network_status(contract, users, ui)
    show_network_graph(contract, users, ui)
except Exception as e:
    print(f"Vouch failed: {e}")

### Receiving a Vouch

Now, let's have another user, 'Bob', also vouch for 'Newbie'. This will change 'Newbie's rank and score.

In [ ]:
# Get Bob's address (the second user)
bob_addr = list(users.keys())[1]
bob_name = ui._get_display_name(users[bob_addr]['name'])

# 3. Make Bob vouch for Newbie
print(f"Action: {bob_name} is vouching for {newbie_name}...")
try:
    contract.vouch(bob_addr, newbie_addr)
    print("...Vouch successful! Displaying updated network state:")

    # Refresh the UI object
    ui.vouch_target.options = ui._get_vouchable_users()
    ui.unvouch_target.options = ui._get_unvouchable_users()
    ui._update_all()
    
    # Show the current state
    show_network_status(contract, users, ui)
    show_network_graph(contract, users, ui)
except Exception as e:
    print(f"Vouch failed: {e}")

### Vouching Back

A vouch can also be reciprocated. Let's have 'Newbie' vouch back for 'Alice'.

In [ ]:
# 4. Make Newbie vouch for Alice
print(f"Action: {newbie_name} is vouching for {alice_name}...")
try:
    contract.vouch(newbie_addr, alice_addr)
    print("...Vouch successful! Displaying updated network state:")

    # Refresh the UI object
    ui.vouch_target.options = ui._get_vouchable_users()
    ui.unvouch_target.options = ui._get_unvouchable_users()
    ui._update_all()
    
    # Show the current state
    show_network_status(contract, users, ui)
    show_network_graph(contract, users, ui)
except Exception as e:
    print(f"Vouch failed: {e}")

### Unvouching

Relationships can change. Let's say 'Alice' decides to unvouch for 'Newbie'.

In [ ]:
# 5. Make Alice unvouch for Newbie
print(f"Action: {alice_name} is unvouching for {newbie_name}...")
try:
    contract.unvouch(alice_addr, newbie_addr)
    print("...Unvouch successful! Displaying updated network state:")

    # Refresh the UI object
    ui.vouch_target.options = ui._get_vouchable_users()
    ui.unvouch_target.options = ui._get_unvouchable_users()
    ui._update_all()
    
    # Show the current state
    show_network_status(contract, users, ui)
    show_network_graph(contract, users, ui)
except Exception as e:
    print(f"Vouch failed: {e}")

## Step 4: Launch Interactive UI

Now that we have programmatically guided you through the main operations, let's display the UI.

You can now freely interact with the network in its current state. The UI below reflects all the changes we just scripted in Step 3.

In [ ]:
print("\n" + "=" * 60)
print("Displaying UI...")
print("=" * 60)

ui.display()